In [1]:
!pip -q install langchain openai huggingface_hub ipywidgets

In [2]:
!nvidia-smi

Wed Jun 28 15:33:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         Off  | 00000000:00:1E.0 Off |                    0 |
|  0%   26C    P8    24W / 300W |      0MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Langchain

LangChain is a framework designed to simplify the creation of applications using large language models.

https://github.com/hwchase17/langchain

## Interact with foundation models directly

In [3]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003",
             temperature=0,
             max_tokens=256)

In [4]:
print(llm("Why did the chicken cross the road?"))



To get to the other side.


## Tuning foundation model with prompts

In [7]:
from langchain import PromptTemplate

restaraunt_template = """
I want you to act as a naming consultant for new restaraunts.

Return a list of restaraunt names. Each name should be short, catchy and easy to remember.

What are some good names for a restaraunt that is {restaraunt_description}?
"""

prompt = PromptTemplate(
    input_variables=["restaraunt_description"],
    template=restaraunt_template
)

In [8]:
print(llm(
    prompt.format(restaraunt_description="roadside chicken wings")
))


1. Wing Stop
2. Wingin' Out
3. Wing-A-Ling
4. Wing-It
5. Wing-Mania
6. Wing-Tastic
7. Wing-Fling
8. Wing-Rama
9. Wing-Ding
10. Wing-A-Roo


## Chaining

In [9]:
llm = OpenAI(model_name="text-davinci-003",
             temperature=0.9,
             max_tokens=256)

In [10]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

template = "What is a good name for a company that makes {product}?"
prompt = PromptTemplate(
    input_variables=["product"],
    template=template,
)

prompt.format(product="colorful socks")
chain = LLMChain(llm = llm, prompt = prompt)
print(chain.run("colorful socks"))



Painted Toes Socks.


In [11]:
from langchain.chains import LLMChain, SimpleSequentialChain

# Create a second chain with a prompt template and an LLM
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a funny catchphrase for the following company: {company_name}",
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine the first and the second chain 
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
catchphrase = overall_chain.run("chickens")



> Entering new  chain...


Cluck Co.


"Clucking up this world one chicken at a time!"

> Finished chain.


## Agents: Accessing other tools

In [12]:
!pip -q install wikipedia

In [13]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

llm = OpenAI(model_name="text-davinci-003",
             temperature=0,
             max_tokens=256)

tools = load_tools(["wikipedia", "llm-math"], llm=llm)
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True)


agent.run("When was Jeff Bezos born? How old was he in 2022?")



> Entering new  chain...
 I need to find out when Jeff Bezos was born and then calculate his age in 2022.
Action: Wikipedia
Action Input: Jeff Bezos
Observation: Page: Jeff Bezos
Summary: Jeffrey Preston Bezos ( BAY-zohss; né Jorgensen; born January 12, 1964) is an American entrepreneur, media proprietor, investor, and commercial astronaut. He is the founder, executive chairman, and former president and CEO of Amazon, the world's largest e-commerce and cloud computing company. With a net worth of US$125 billion as of April 2023, Bezos is the third-wealthiest person in the world and was the wealthiest from 2017 to 2021, according to both the Bloomberg Billionaires Index and Forbes.Born in Albuquerque and raised in Houston and Miami, Bezos graduated from Princeton University in 1986. He holds a degree in electrical engineering and computer science. He worked on Wall Street in a variety of related fields from 1986 to early 1994. Bezos founded Amazon in late 1994 on a road trip from New 

'Jeff Bezos was born on January 12, 1964, and was 58 years old in 2022.'